In [1]:
import numpy as np 
import pandas as pd 
import json
from pandas.io.json import json_normalize
import functools
from multiprocessing import Pool
import logging
import gc
import matplotlib.pyplot as plt
import time
from scipy.stats import stats
from ast import literal_eval
import warnings
import os
from pandas.core.common import SettingWithCopyWarning
import ast
import pandas.io.json as pdjson
warnings.simplefilter('error', SettingWithCopyWarning)
gc.enable()
%matplotlib inline

In [2]:
path='/home/baitong/pywork/RevenuePrediction/all (1)/train_v2.csv'

In [10]:
gc.enable()
def load_df(csv_path,n):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                    chunksize = 100000)
    count = 0
    for df in dfs:
        count+=1
        if count==n:
            df.reset_index(drop = True,inplace = True)       
            for column in JSON_COLUMNS:
                column_as_df = json_normalize(df[column])
                column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
                df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

            print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
            use_df = df
            del df
            gc.collect()
            ans = pd.concat([ans, use_df], axis = 0).reset_index(drop = True)
            print(ans.shape)
            break
        else :
            continue
    return ans


In [11]:
def parseData(df):
    df['hits']=df['hits'].apply(ast.literal_eval)
    df['hits']=df['hits'].str[0]
    df['hits']=df['hits'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)
    
    df['customDimensions']=df['customDimensions'].apply(ast.literal_eval)
    df['customDimensions']=df['customDimensions'].str[0]
    df['customDimensions']=df['customDimensions'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)
    
    JSON_COLUMNS = ['hits','customDimensions']
    for column in JSON_COLUMNS:
        column_as_df = pdjson.json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        print("'parse' function to flatten JSON columns :",column)
    return df

In [12]:
# 
useless_feature1 = ['trafficSource_adwordsClickInfo.gclId','hits_contentGroup.previousContentGroup1','hits_contentGroup.previousContentGroup2',
'hits_contentGroup.previousContentGroup3','hits_contentGroup.previousContentGroup4',
'hits_contentGroup.previousContentGroup5','hits_customDimensions',
'hits_customMetrics','hits_customVariables',            
                  ]
# ['hits_latencyTracking.domainLookupTime'\n 'hits_latencyTracking.redirectionTime'\n 'hits_latencyTracking.serverConnectionTime']
useless_feature2 = ['hits_latencyTracking.domContentLoadedTime',
'hits_latencyTracking.domInteractiveTime',
'hits_latencyTracking.domLatencyMetricsSample',
'hits_latencyTracking.pageDownloadTime',
'hits_latencyTracking.pageLoadSample',
'hits_latencyTracking.pageLoadTime',
'hits_latencyTracking.serverResponseTime',
'hits_latencyTracking.speedMetricsSample']
useless_feature3 = ['hits_publisher_infos','hits_contentGroup.contentGroup1',
'hits_contentGroup.contentGroup2',
'hits_contentGroup.contentGroup3',
'hits_contentGroup.contentGroup4',
'hits_contentGroup.contentGroup5',
'hits_contentGroup.contentGroupUniqueViews1',
'hits_contentGroup.contentGroupUniqueViews2',
'hits_contentGroup.contentGroupUniqueViews3']
useless_feature4 = ['hits_eventInfo.eventAction','hits_eventInfo.eventCategory','hits_eventInfo.eventLabel',
                   'hits_experiment'
                  ]
useless_feature = useless_feature1+useless_feature2+useless_feature3+useless_feature4

In [13]:
def featureSelect(df):
    #获取不变的常量列，模型无法在常量数据计学到东西，数据与处理时需要drop
    const_cols = [c for c in df.columns if df[c].nunique(dropna=False)==1 ]
#     print(const_cols)    
    df = df.drop(const_cols , axis=1)
    df = parseData(df)
    df = df.drop(labels=['hits_product'], axis=1)
    df = df.drop(labels=['hits_promotion'], axis=1)
    df = df.drop(labels=useless_feature, axis=1)
    return df
    

In [ ]:
for i in range(18):
    if i >= 5:
        print("loading chunksize %d / 18 :"%(i+1))
        train = load_df(path,i+1)
        train = featureSelect(train)
        train.to_csv("/home/baitong/pywork/RevenuePrediction/train0" +str(i) +'.csv',index = False)
        print("loaded chunksize %d / 18 :"%(i+1))
        print('-'*50)

loading chunksize 6 / 18 :
Loaded train_v2.csv. Shape: (100000, 59)
(100000, 59)
'parse' function to flatten JSON columns : hits
'parse' function to flatten JSON columns : customDimensions
loaded chunksize 6 / 18 :
--------------------------------------------------
loading chunksize 7 / 18 :
Loaded train_v2.csv. Shape: (100000, 59)
(100000, 59)
'parse' function to flatten JSON columns : hits
'parse' function to flatten JSON columns : customDimensions
loaded chunksize 7 / 18 :
--------------------------------------------------
loading chunksize 8 / 18 :
Loaded train_v2.csv. Shape: (100000, 59)
(100000, 59)
'parse' function to flatten JSON columns : hits
'parse' function to flatten JSON columns : customDimensions
loaded chunksize 8 / 18 :
--------------------------------------------------
loading chunksize 9 / 18 :
Loaded train_v2.csv. Shape: (100000, 59)
(100000, 59)
'parse' function to flatten JSON columns : hits
'parse' function to flatten JSON columns : customDimensions
loaded chunks

In [ ]:
train

In [ ]:
# jsonlist=[]
# for i in range(len(train.columns)):   # for each column
#     if (isinstance(train.iloc[1,i], list) ):  # see if some element 1 is a list
#         jsonlist.append( train.columns[i] )   # if yes, then save name to list
# print(jsonlist)

In [ ]:
# print("Printout for each column's number of unique values (incl. nans)\n")
# for col in train.columns:
#     try:
#         print(col, ':', train[col].nunique(dropna=False))
#     except TypeError:
#         a=train[col].astype('str')
#         #print(a)
#         print( col, ':', a.nunique(dropna=False), ' >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> LIST')
# # Clean workspace
# del(col)

In [ ]:
# print('Data shape before dropping constant columns:', train.shape)

# print('\nColumns being dropped:')

# for col in train.columns:
#     try:
#         if (train[col].nunique(dropna=False) == 1):
#             del(train[col])
#             print(col)
#     except TypeError:
#         a=train[col].astype('str')
#         if (a.nunique(dropna=False) == 1):
#             del(train[col])
#             print(col)
# del(col)

# print('\ndata shape is now:', train.shape)

In [ ]:
train.info()

In [ ]:
print("Printout for each column's number of unique values (incl. nans)\n")
for col in train.columns:
    try:
        print(col)
    except TypeError:
        a=train[col].astype('str')
        #print(a)
        print( col)
# Clean workspace
del(col)